### GENERATE A RANDOM BPMN

In [ ]:
#################
# IMPORTS
#################
import requests
import re
import random
import sys
sys.path.append('src')
from paco.parser.parse_tree import ParseTree
from paco.execution_tree.execution_tree import ExecutionTree
from paco.explainer.bdd.bdds import bdds_from_json

#################
# HEADERS
#################
headers = {
	"Content-Type": "application/json",
}
url = "http://127.0.0.1:8000/"

### Define and Check BPMN

In [ ]:
from utils.env import EXPRESSION, IMPACTS, DURATIONS, IMPACTS_NAMES, PROBABILITIES, DELAYS, NAMES, LOOP_PROBABILITY, LOOP_ROUND, H
import graphviz
from IPython.display import display, SVG

expression = '(((((((T1,T2)/[C1]((T3,T4)||T5)),((T6,T7)^[N1]T8)),((T9/[C2]T10),(T11,((T12,T13),T14)))),(((T15/[C3]T16)^[N3]T17)^[N2](T18,T19)))/[C4]((((T20,T21),T22)||T23),((T24,T25)/[C5]T26)))||((T27||((T28^[N4]T29)^[N5](T30/[C6](((T31,T32),((T33^[N7]T34)/[C7]T35)),(T36,T37)))))||T38))'

impacts_names = ["cost", "CO2"]

impacts_range = [1, 50]
duration_range = [1, 100]
delay_range = [0, 10]

# Consider a task each T
tasks = sorted(set(re.findall(r'T\d+', expression)))
natures = sorted(set(re.findall(r'N\d+', expression)))
choices = sorted(set(re.findall(r'C\d+', expression)))
bpmn = {
	EXPRESSION: expression,
	IMPACTS: {task: [random.randint(impacts_range[0], impacts_range[1]) for _ in impacts_names] for task in tasks},
	DURATIONS: {task: [1, random.randint(duration_range[0], duration_range[1])] for task in tasks},
	IMPACTS_NAMES: impacts_names,
	DELAYS: {choice: random.randint(delay_range[0], delay_range[1]) for choice in choices},
	PROBABILITIES: {nature: round(random.uniform(0.1, 0.9), 2) for nature in natures},
	NAMES: {choice: choice for choice in choices} | {nature: nature for nature in natures},
	LOOP_PROBABILITY : {}, LOOP_ROUND: {}, H: 0,
}

try:
	resp = requests.get(f'{url}create_bpmn', json={'bpmn': bpmn},  headers=headers)
	resp.raise_for_status()
	display(SVG(graphviz.Source(resp.json()['bpmn_dot']).pipe(format="svg")))

except requests.exceptions.HTTPError as e:
	print(f"HTTP Error ({resp.status_code}):", resp.json())

### Sample Expected Impact

In [ ]:
try:
	response = requests.get(f'{url}create_parse_tree', json={"bpmn": bpmn},  headers=headers)
	resp.raise_for_status()

	parse_tree, pending_choices, pending_natures = ParseTree.from_json(
		response.json()['parse_tree'],
		impact_size=len(bpmn['impacts_names']),
		non_cumulative_impact_size=0)

	bound = parse_tree.sample_expected_impact()
	print(f"Sampled Expected Impact: {bound}")

	display(SVG(graphviz.Source(parse_tree.to_dot()).pipe(format="svg")))

except requests.exceptions.HTTPError as e:
	print(f"HTTP Error ({resp.status_code}):", resp.json())

### Create the Execution tree

In [ ]:
try:
	resp = requests.get(f'{url}create_execution_tree', json={"bpmn": bpmn}, headers=headers)
	resp.raise_for_status()

	response = resp.json()
	execution_tree = ExecutionTree.from_json(parse_tree, response["execution_tree"], bpmn["impacts_names"])
	dot = execution_tree.to_dot(state=True, executed_time=True, diff=True)
	display(SVG(graphviz.Source(dot).pipe(format="svg")))

except requests.exceptions.HTTPError as e:
	print(f"HTTP Error ({resp.status_code}):", resp.json())

### Search using Refinements Algorithm
#### Create Execution Tree
#### Found Strategy
#### Explain Strategy

In [ ]:
def search(bpmn, bound, parse_tree, execution_tree, search_only=False):
	resp = requests.get(f'{url}search_only_strategy', json={"bpmn": bpmn, "bound": bound, "parse_tree": parse_tree.to_dict(), "execution_tree": execution_tree.to_dict(), "search_only": search_only}, headers=headers)

	resp.raise_for_status()

	response = resp.json()
	if "frontier_solution" in response:
		return True, response
	else:
		return False, response

In [ ]:
def found_strategy(bpmn, parse_tree, initial_bounds, num_refinements = 10):
	intervals = [ [0.0, bound_value] for bound_value in initial_bounds ]
	bounds = []
	for i in range(len(intervals)):
		bounds.append(intervals[i][1])

	for iteration in range(num_refinements):
		for current_impact in range(len(intervals)):
			test_bounds = []
			for i in range(len(intervals)):
				test_bounds.append(intervals[i][1])

			test_bounds[current_impact] = (intervals[current_impact][0] + intervals[current_impact][1]) / 2

			success, result = search(bpmn, test_bounds, parse_tree, execution_tree, search_only=True)
			if success:
				intervals[current_impact][1] = (intervals[current_impact][0] + intervals[current_impact][1]) / 2
				bounds = test_bounds
			else:
				intervals[current_impact][0] = (intervals[current_impact][0] + intervals[current_impact][1]) / 2


	success, result = search(bpmn, bounds, parse_tree, execution_tree)
	print("Search: ", result["result"])
	if not success:
		raise Exception("No solution found, bounds: " + str(bounds))

	if "strategy_tree" in result:
		explained_choices = bdds_from_json(parse_tree, result["bdds"])
		print("1 is dashed line of BPMN or Parse Tree")
		for choice, bdd in explained_choices.items():
			print(f"{choice.name} : {bdd.typeStrategy}")
			svg_data = graphviz.Source(bdd.bdd_to_dot()).pipe(format="svg")
			display(SVG(svg_data))

		strategy_tree = ExecutionTree.from_json(parse_tree, result["strategy_tree"], bpmn["impacts_names"], explained_choices)

		dot = strategy_tree.to_dot(state=True, executed_time=False, diff=True)
		display(SVG(graphviz.Source(dot).pipe(format="svg")))

	return result


try:
	result = found_strategy(bpmn, parse_tree, bound, num_refinements=1)
except requests.exceptions.HTTPError as e:
	print(f"HTTP Error ({resp.status_code}):", resp.json())